# CS448 - Lab 2: Filter Design and Usage

For this lab you will learn how to design some simple filters and how to apply them to solve some common audio problems. Python’s scipy.signal package has an extensive set of commands to help you design filters (firwin, firwin2, butter, cheby1, cheby2,  ellip, …), so there is no shortage of options.

## Part 1: When to use what 

There will be four cases to this part, each requiring a different type of filter to address a problem in a recording.  The input recordings are:

- ```case1.wav``` [https://drive.google.com/uc?export=download&id=1eTsDiXqqLQv3murPz25O54E89i3DL__d ] : A noise-corrupted speech signal. We want to eliminate the noise.
- ```case2.wav``` [https://drive.google.com/uc?export=download&id=1egd22CxPUe6sINIi0FPTbMfG4S_In2hT ] : Same as above, different type of noise. We want to remove the noise again.
- ```case3.wav``` [https://drive.google.com/uc?export=download&id=1eF-VOVWoT1rh1wAC06WT1ANusMKyDYSn ] : Bird songs during a thunderstorm. As a world renowned ornithologist you need to have a cleaner recording of the bird songs for further analysis.
- ```case4.wav``` [https://drive.google.com/uc?export=download&id=1eeizGhrBICf6pW5OXcbq7ChF4m2N6yIk ]: The signal that we require to extract here is a Morse code which is buried in environmental noise. Design a filter to bring out the beeps.

For each case do the following:
- Plot the spectrogram of the given sound and identify the problem
- Describe what kind of filter will address this problem and why
- Design an FIR filter using ```scipy.signal.firwin()``` and/or ```scipy.signal.firwin2()```
- Design an FIR filter using the formulas in my slides (i.e. do not use ```scipy.signal```)
- Design an IIR Butterworth filter using ```scipy.signal```'s routines
- Show a plot comparing the response of all the filters (hint: ```scipy.signal.freqz```)
- Plot spectrograms of the filtered recordings and compare with the input to show that it fixed the problem
- Play the filtered sounds so that we can hear how well it works

Make some observations on how the results differ between an FIR and IIR filter and try to find the best possible filter size/type/parameters to produce the best result. Show results under various parameters (e.g. filter length) and make some plots that demonstrate the effects of these parameters. Most importantly, try to get a sense of how these design choices impact audible quality. Being able to listen at a sound and identify what’s wrong and how to fix it is a big part of audio processing.

Hint: To apply an FIR filter you can use ```scipy.signal.convolve```, to apply an IIR filter (or an FIR) you can use ```scipy,signal.lfilter```.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from scipy.signal import firwin, freqz, lfilter, get_window, butter, convolve

In [ ]:
### UTILITIES ###

# Sound player function that plays array "x" with a sample rate "rate", and labels it with "label"
def sound(x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    display(
        HTML('<style> table, th, td {border: 0px; }</style> <table><tr><td>' +
             label + '</td><td>' + Audio(x, rate=rate)._repr_html_()[3:] +
             '</td></tr></table>'))

# Function that normalizes a signal
def normalize_signal(x):
    return x / np.max(np.abs(x))

# Function that plots the spectrogram of a sound
def plot_spectrogram(input_sound, fs, title="Spectrogram"):
    plt.title(title)
    plt.specgram(input_sound, Fs=fs, cmap="winter")
    plt.xlabel('Time [sec]')
    plt.ylabel('Frequency [Hz]')
    plt.show()

# Function that plots the frequency response of a filter
def plot_freq_response(w,
                       h,
                       fs,
                       cutoff,
                       title):
    plt.plot(0.5 * fs * w / np.pi, np.abs(h), 'b')
    plt.axvline(cutoff, color='k', linestyle='--')
    plt.xlim(0, 0.5 * fs)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain')
    plt.title(title)
    plt.grid(True)
    plt.show()


In [ ]:
### LOW-PASS FILTERS ###
# Function that creates a firwin low-pass filter
def firwin_low_pass(input, fs, cutoff):
    firwin_case1 = firwin(5000, cutoff, fs=fs, pass_zero=True)
    w, h = freqz(firwin_case1, worN=8000)

    return [convolve(firwin_case1, input), w, h]


# Function that creates an FIR low-pass filter
def FIR_low_pass(input, cutoff):
    FIR = cutoff * np.sinc(cutoff * np.arange(-200, 200))
    FIR_windowed = np.multiply(FIR, get_window('hann', 400))

    w, h = freqz(FIR_windowed, worN=8000)
    return [convolve(FIR_windowed, input), w, h]


# Function that creates a butterworth low-pass filter
def butter_low_pass(input, fs, cutoff, order=12):
    b, a = butter(order, cutoff / (fs / 2))
    w, h = freqz(b, a)

    return [lfilter(b, a, input), w, h]

### HIGH-PASS FILTERS ###
# Function that creates a firwin high-pass filter
def firwin_high_pass(input, fs, cutoff):
    firwin_case1 = firwin(5000, cutoff, fs=fs, pass_zero=True)
    w, h = freqz(firwin_case1, worN=8000)

    return [convolve(firwin_case1, input), w, h]


# Function that creates an FIR high-pass filter
def FIR_high_pass(input, cutoff):
    FIR = cutoff * np.sinc(cutoff * np.arange(-200, 200))
    FIR_windowed = np.multiply(FIR, get_window('hann', 400))

    w, h = freqz(FIR_windowed, worN=8000)
    return [convolve(FIR_windowed, input), w, h]


# Function that creates a butterworth high-pass filter
def butter_high_pass(input, fs, cutoff, order=12):
    b, a = butter(order, cutoff / (fs / 2))
    w, h = freqz(b, a)

    return [lfilter(b, a, input), w, h]

In [ ]:
### Case 1 ###

# Load the sound, play it, and plot it's spectrogram
fs, input_sound = wavfile.read("./data/case1.wav")
input_sound = normalize_signal(input_sound)

sound(input_sound, rate=fs, label='case1.wav')
plot_spectrogram(input_sound, fs, title="Spectrogram of case1.wav")

# Design a filter that fixes things
### A low-pass filter with a cutoff frequency around 6KHz would be good here as the
### noise present in the recording seems to be the only data above that threshold. I chose
### the exact frequency 5700Hz after a process of trial and error.
# Filter using firwin and filtering the input sound
cutoff = 5700
firwin_sound, w, h = firwin_low_pass(input_sound, fs, cutoff)

# Plot the resulting spectrogram, frequency response, and play the resulting sound
sound(firwin_sound, rate=fs, label='case1.wav after filtering with firwin')

plot_freq_response(w,
                   h,
                   fs,
                   cutoff,
                   title="Frequency response of the firwin low-pass filter")

plot_spectrogram(firwin_sound,
                 fs,
                 title="Spectrogram of case1.wav after filtering with firwin")

# Design an FIR filter using the formulas in my slides (i.e. do not use scipy.signal)
cutoff = .35  # I arrived at this cutoff frequency by trial and error
FIR_sound, w, h = FIR_low_pass(input_sound, cutoff)

# Plot the resulting spectrogram, frequency response, and play the resulting sound
sound(FIR_sound, rate=fs, label='case1.wav after filtering with FIR')
plot_freq_response(w,
                   h,
                   fs,
                   cutoff * fs / 2,
                   title="Frequency response of the FIR low-pass filter")

plot_spectrogram(FIR_sound,
                 fs,
                 title="Spectrogram of case1.wav after filtering with FIR")

# Design an IIR Butterworth filter using scipy.signal's routines
cutoff = 5000  # I arrived at this cutoff frequency & order by trial and error

# Filter the input sound
butterworth_sound, w, h = butter_low_pass(input_sound, fs, cutoff)

# Plot the resulting spectrogram, frequency response, and play the resulting sound
sound(butterworth_sound,
      rate=fs,
      label='case1.wav after filtering with IIR Butterworth')

plot_freq_response(
    w,
    h,
    fs,
    cutoff,
    title="Frequency response of the IIR low-pass Butterworth filter")

plot_spectrogram(
    butterworth_sound,
    fs,
    title="Spectrogram of case1.wav after filtering with IIR Butterworth")

In [ ]:
### Case 2 ###

# Load the sound, play it, and plot it's spectrogram
fs_case2, input_sound_case2 = wavfile.read("./data/case2.wav")
input_sound_case2 = normalize_signal(input_sound_case2)

sound(input_sound_case2, rate=fs_case2, label='case2.wav')
plot_spectrogram(input_sound_case2, fs_case2, title="Spectrogram of case2.wav")

# Design a filter that fixes things
### A band-reject filter with the rejected band being around 2KHz - 2.7Khz would be
### good here as there that is where the telephone's frequencies seem to reside.
### Removing this should lead to a much cleaner audio file of the speech.
# Filter using firwin and filtering the input sound
cutoff = [1800, 2900]
firwin_low_cutoff_case1 = 1800
firwin_high_cutoff_case2 = 2900

firwin_case2 = firwin(201, cutoff,
                      fs=fs_case2)
firwin_sound_case2 = convolve(firwin_case2, input_sound_case2)

# Plot the resulting spectrogram, frequency response, and play the resulting sound
sound(firwin_sound_case2,
      rate=fs_case2,
      label='case2.wav after filtering with firwin')

w, h = freqz(firwin_case2, worN=8000)
plot_freq_response(w,
                   h,
                   fs_case2,
                   (firwin_low_cutoff_case1 + firwin_high_cutoff_case2) / 2,
                   title="Frequency response of the firwin band-reject filter")

plot_spectrogram(firwin_sound_case2,
                 fs_case2,
                 title="Spectrogram of case2.wav after filtering with firwin")

# Design an FIR filter using the formulas in my slides (i.e. do not use scipy.signal)
FIR_band_width_case2 = 800 / fs_case2 * 2
FIR_band_center_case2 = 2400 / fs_case2 * 2
FIR_N = 1000

n = np.arange(-FIR_N, FIR_N)
FIR_case2 = -2 * np.cos(
    np.pi * n * FIR_band_center_case2) * FIR_band_width_case2 * np.sinc(
        FIR_band_width_case2 * n)
FIR_case2[int(len(FIR_case2) / 2)] += 1

FIR_windowed_case2 = np.multiply(FIR_case2, get_window('hann', FIR_N * 2))
FIR_sound_case2 = convolve(FIR_windowed_case2, input_sound_case2)

# Plot the resulting spectrogram, frequency response, and play the resulting sound
sound(FIR_sound_case2,
      rate=fs_case2,
      label='case2.wav after filtering with FIR')

w, h = freqz(FIR_windowed_case2, worN=8000)
plot_freq_response(w,
                   h,
                   fs_case2,
                   FIR_band_center_case2 * fs_case2 / 2,
                   title="Frequency response of the FIR band-reject filter")

plot_spectrogram(FIR_sound_case2,
                 fs_case2,
                 title="Spectrogram of case2.wav after filtering with FIR")

# Design an IIR Butterworth filter using scipy.signal's routines
butterworth_low_cutoff_case2 = 1600
butterworth_high_cutoff_case2 = 3100
butterworth_order_case2 = 12
b, a = butter(butterworth_order_case2, [
    butterworth_low_cutoff_case2 / fs_case2 * 2,
    butterworth_high_cutoff_case2 / fs_case2 * 2
],
              btype='bandstop')

# Filter the input sound
butterworth_sound_case2 = lfilter(b, a, input_sound_case2)

# Plot the resulting spectrogram, frequency response, and play the resulting sound
sound(butterworth_sound_case2,
      rate=fs_case2,
      label='case2.wav after filtering with IIR Butterworth')

w, h = freqz(b, a)
plot_freq_response(
    w,
    h,
    fs_case2,
    (butterworth_low_cutoff_case2 + butterworth_high_cutoff_case2) / 2,
    title="Frequency response of the IIR band-reject Butterworth filter")

plot_spectrogram(
    butterworth_sound_case2,
    fs_case2,
    title="Spectrogram of case2.wav after filtering with IIR Butterworth")


In [ ]:
### Case 3 ###

# Load the sound, play it, and plot it's spectrogram
fs_case3, input_sound_case3 = wavfile.read("./data/case3.wav")
input_sound_case3 = normalize_signal(input_sound_case3)

sound(input_sound_case3, rate=fs_case3, label='case3.wav')
plot_spectrogram(input_sound_case3, fs_case3, title="Spectrogram of case3.wav")

# Design a filter that fixes things
### A high-pass filter with a cutoff frequency around 2KHz would be good here as there
### is a lot of low-frequency muddiness in the recording. Removing this should lead to
### a much cleaner sound of just the birds.
# Filter using firwin and filtering the input sound
firwin_cutoff_case3 = 2000

firwin_case3 = firwin(201, firwin_cutoff_case3, fs=fs_case3, pass_zero=False)
firwin_sound_case3 = convolve(firwin_case3, input_sound_case3)

# Plot the resulting spectrogram, frequency response, and play the resulting sound
sound(firwin_sound_case3,
      rate=fs_case3,
      label='case3.wav after filtering with firwin')

w, h = freqz(firwin_case3, worN=8000)
plot_freq_response(w,
                   h,
                   fs_case3,
                   firwin_cutoff_case3,
                   title="Frequency response of the firwin high-pass filter")

plot_spectrogram(firwin_sound_case3,
                 fs_case3,
                 title="Spectrogram of case3.wav after filtering with firwin")

# Design an FIR filter using the formulas in my slides (i.e. do not use scipy.signal)
FIR_cutoff_case3 = .25  # I arrived at this cutoff frequency by trial and error
FIR_N = 200

FIR_case3 = -FIR_cutoff_case3 * np.sinc(
    FIR_cutoff_case3 * np.arange(-200, 200))
FIR_case3[int(len(FIR_case3) / 2)] += 1

FIR_windowed_case3 = np.multiply(FIR_case3, get_window('hann', 400))
FIR_sound_case3 = convolve(FIR_windowed_case3, input_sound_case3)

# Plot the resulting spectrogram, frequency response, and play the resulting sound
sound(FIR_sound_case3,
      rate=fs_case3,
      label='case3.wav after filtering with FIR')

w, h = freqz(FIR_windowed_case3, worN=8000)
plot_freq_response(w,
                   h,
                   fs_case3,
                   FIR_cutoff_case3 * fs_case3 / 2,
                   title="Frequency response of the FIR high-pass filter")

plot_spectrogram(FIR_sound_case3,
                 fs_case3,
                 title="Spectrogram of case3.wav after filtering with FIR")

# Design an IIR Butterworth filter using scipy.signal's routines
butterworth_cutoff_case3 = 2000  # I arrived at this cutoff frequency & order by trial and error
butterworth_order_case3 = 12
b, a = butter(butterworth_order_case3,
              butterworth_cutoff_case3,
              'highpass',
              fs=fs_case3)

# Filter the input sound
butterworth_sound_case3 = lfilter(b, a, input_sound_case3)

# Plot the resulting spectrogram, frequency response, and play the resulting sound
sound(butterworth_sound_case3,
      rate=fs_case3,
      label='case3.wav after filtering with IIR Butterworth')

w, h = freqz(b, a)
plot_freq_response(
    w,
    h,
    fs_case3,
    butterworth_cutoff_case3,
    title="Frequency response of the IIR high-pass Butterworth filter")

plot_spectrogram(
    butterworth_sound_case3,
    fs_case3,
    title="Spectrogram of case3.wav after filtering with IIR Butterworth")

In [15]:
# Load the sound, play it, and plot it's spectrogram
fs_case4, input_sound_case4 = wavfile.read("./data/case4.wav")
input_sound_case4 = normalize_signal(input_sound_case4)
print(800 / fs * 2, (.1 * fs) / 2)

sound(input_sound_case4, rate=fs_case4, label='case4.wav')
# plot_spectrogram(input_sound_case4, fs_case4, title="Spectrogram of case4.wav")

# # Design a filter that fixes things
# ### A band-pass filter with the pass band being from 800Hz - 1000Hz would be
# ### good here as there that is where the morse code's frequencies seem to reside.
# ### Keeping only this should lead to a much cleaner recording of the message.
# # Filter using firwin and filtering the input sound
# firwin_cutoff_case4 = np.array([800, 1000]) / (fs_case4 / 2)

# firwin_case4 = firwin(200, firwin_cutoff_case4, pass_zero=False)
# firwin_sound_case4 = convolve(firwin_case4, input_sound_case4)

# # Plot the resulting spectrogram, frequency response, and play the resulting sound
# sound(firwin_sound_case4,
#       rate=fs_case4,
#       label='case4.wav after filtering with firwin')

# w, h = freqz(firwin_case4, worN=8000)
# plot_freq_response(w,
#                    h,
#                    fs_case4, (800 + 1000) / 2,
#                    title="Frequency response of the firwin band-pass filter")

# plot_spectrogram(firwin_sound_case4,
#                  fs_case4,
#                  title="Spectrogram of case4.wav after filtering with firwin")

# # Design an FIR filter using the formulas in my slides (i.e. do not use scipy.signal)
# FIR_band_width_case4 = .01
# FIR_band_center_case4 = 0.1
# FIR_N = 1000

# n = np.arange(-FIR_N, FIR_N)
# FIR_case4 = 2 * np.cos(
#     np.pi * n * FIR_band_center_case4) * FIR_band_width_case4 * np.sinc(
#         FIR_band_width_case4 * n)

# FIR_windowed_case4 = np.multiply(FIR_case4, get_window('hann', FIR_N * 2))
# FIR_sound_case4 = convolve(FIR_windowed_case4, input_sound_case4)

# # Plot the resulting spectrogram, frequency response, and play the resulting sound
# sound(FIR_sound_case4,
#       rate=fs_case4,
#       label='case4.wav after filtering with FIR')

# w, h = freqz(FIR_windowed_case4, worN=8000)
# plot_freq_response(w,
#                    h,
#                    fs_case4,
#                    FIR_band_center_case4 * fs_case4 / 2,
#                    title="Frequency response of the FIR band-pass filter")

# plot_spectrogram(FIR_sound_case4,
#                  fs_case4,
#                  title="Spectrogram of case4.wav after filtering with FIR")

# # Design an IIR Butterworth filter using scipy.signal's routines
# butterworth_low_cutoff_case4 = 800
# butterworth_high_cutoff_case4 = 1000
# butterworth_order_case4 = 5

# b, a = butter(butterworth_order_case4, [
#     butterworth_low_cutoff_case4 / fs_case4 * 2,
#     butterworth_high_cutoff_case4 / fs_case4 * 2
# ],
#               btype='band')

# # Filter the input sound
# butterworth_sound_case4 = lfilter(b, a, input_sound_case4)

# # Plot the resulting spectrogram, frequency response, and play the resulting sound
# sound(butterworth_sound_case4,
#       rate=fs_case4,
#       label='case4.wav after filtering with IIR Butterworth')

# w, h = freqz(b, a)
# plot_freq_response(
#     w,
#     h,
#     fs_case4,
#     (butterworth_low_cutoff_case4 + butterworth_high_cutoff_case4) / 2,
#     title="Frequency response of the IIR band-pass Butterworth filter")

# plot_spectrogram(
#     butterworth_sound_case4,
#     fs_case4,
#     title="Spectrogram of case4.wav after filtering with IIR Butterworth")


0.05 1600.0


case4.wav,Your browser does not support the audio element.


## Part 2. Designing a simple equalizer

For this part we will design a simple graphic equalizer. We will do so using a more straightforward approach as opposed to a bank of filters as discussed in class.

We want to make an equalizer which contains six bands with center frequencies at 100Hz, 200Hz, 400Hz, 800Hz, 1600Hz and 3200Hz. Your equalizer function will take two inputs, one for the input sound and a 6-element gain vector that will indicate how much to boost or suppress each frequency band. Use the ```scipy.signal.firwin2``` function to design a filter that has the desired characteristics. For various settings of the gain vector, use the ```scipy.signal.freqz``` command to plot the response of the filter and verify that it behaves as indicated. Experiment with various filter lengths and see which works best.

Once you figure that out, design a graphic equalizer with as many bands as you like (and arbitrary center frequencies as well), and use it to solve the problems in part 1 again. The only thing that should be different in the EQ for each recording should be the gains for each band. Play the output sounds, and show the spectrograms, see how they compare with your previous solutions.

Optional extra credit (+1pt): Use ipywidgets to make interactive sliders and process an audio stream and play it from the speakers in real-time (either from mic input, or just stream audio from disk).

In [ ]:
# Design an equalizer function
def equalizer(input_sound, gains):
    # YOUR CODE HERE
    raise NotImplementedError()


# Show its response with various gain settings
# YOUR CODE HERE
raise NotImplementedError()

# Show how it can denoise the examples in part 1
# YOUR CODE HERE
raise NotImplementedError()